In [1]:
# Pip install scikeras and python-Levenshtein, uncomment them and restart the
# session, pip install fuzzywuzzy, uncomment and restart, and then
# everything should be able to run

import numpy as np
import random, time, sys

# data processing
import pandas as pd
import json


#!pip install --upgrade tensorflow
import tensorflow as tf
from tensorflow import keras
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import save_model, load_model

# evaluation and hyperparameter tuning
#!pip install scikeras[tensorflow]
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD
from scipy.stats import uniform

# word processing
#!pip install python-Levenshtein
#!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

In [2]:
# This only became an issue later on in the project when importing modules
# for Hyperparameter tuning
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

2.12.1
2.12.0


In [3]:
# Reading the JSON file
with open('intents.json', 'r') as file:
    intents = json.load(file)

In [4]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [5]:
for tag in intents['intents']:
  for pattern in tag['patterns']:
    print(f"{pattern.lower()}\t{tag['tag']}")

hi there	greeting
how are you	greeting
is anyone there?	greeting
hey	greeting
hola	greeting
hello	greeting
good day	greeting
bye	goodbye
see you later	goodbye
goodbye	goodbye
nice chatting to you, bye	goodbye
till next time	goodbye
thanks	thanks
thank you	thanks
that's helpful	thanks
awesome, thanks	thanks
thanks for helping me	thanks
how you could help me?	options
what you can do?	options
what help you provide?	options
how you can be helpful?	options
what support is offered	options
how to check adverse drug reaction?	adverse_drug
open adverse drugs module	adverse_drug
give me a list of drugs causing adverse behavior	adverse_drug
list all drugs suitable for patient with adverse reaction	adverse_drug
which drugs dont have adverse reaction?	adverse_drug
open blood pressure module	blood_pressure
task related to blood pressure	blood_pressure
blood pressure data entry	blood_pressure
i want to log blood pressure results	blood_pressure
blood pressure data management	blood_pressure
i want to s

In [6]:
# Creating our training data
# We use append mode with the txt in order to create a txt file with this
# name in our working directory
# Since some of our patterns include a comma, we can't use comma as our
# delimiter and we will use a tab "\t" instead
with open('training_data_hospitalBot.txt', "a") as f:
    f.write("patterns\ttags\n")
    for tag in intents['intents']:
      for pattern in tag['patterns']:
        f.write(f"{pattern.lower()}\t{tag['tag']}\n")

In [7]:
training_data = pd.read_csv("/content/training_data_hospitalBot.txt",
                            delimiter='\t')
training_data.drop(index=training_data.index[0], axis=0, inplace=True) # Drop
# the first row which mimics the headings
training_data = training_data.reset_index(drop=True)

In [8]:
training_data

,patterns,tags
0,how are you,greeting
1,is anyone there?,greeting
2,hey,greeting
3,hola,greeting
4,hello,greeting
...,...,...
185,lookup for hospital,hospital_search
186,searching for hospital to transfer patient,hospital_search
187,i want to search hospital data,hospital_search
188,hospital lookup for patient,hospital_search


In [9]:
# Preprocessing training data; transforming it into something that our
# computers can understand
# convert to lowercase so that the bot isn’t distinguishing the case of
# the characters
training_data["patterns"] = training_data["patterns"].str.lower()
vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words="english")
training_data_tfidf = vectorizer.fit_transform(training_data["patterns"]).toarray()

In [10]:
training_data_tfidf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.35056373, 0.        ,
        0.38552431],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
# Preprocessing target variable (tags); this is also used to transform
# categories, in our case our tags, into numerical format
le = LabelEncoder()
training_data_tags_le = pd.DataFrame({"tags": le.fit_transform(training_data["tags"])})
# And additionally we use Dummy Encoding so that there won’t be any intrinsic
# order or priority within the categories. Each category will be treated as an
# independent and equally important feature.
training_data_tags_dummy_encoded = pd.get_dummies(training_data_tags_le["tags"]).to_numpy()

In [12]:
# Creating the DNN
hospitalbot = Sequential()
# The input layer will have 10 nodes and the shape of our training data will
# determine how the input data is entering the neural network
# Since we don’t explicitely type anything, ReLU is the activation function
# we use in the intput layer and our three hidden layers, it’s the default
# function.
hospitalbot.add(Dense(10, input_shape=(len(training_data_tfidf[0]),)))
hospitalbot.add(Dense(8))
hospitalbot.add(Dense(8))
hospitalbot.add(Dense(6))
# And we use softmax as our activation function in the Output layer since
# it’s a multi-class classification problem
hospitalbot.add(Dense(len(training_data_tags_dummy_encoded[0]),
                      activation="softmax"))
hospitalbot.compile(optimizer="rmsprop", loss="categorical_crossentropy",
                    metrics="accuracy")

In [13]:
# Fitting DNN
hospitalbot.fit(training_data_tfidf, training_data_tags_dummy_encoded,
                epochs=100, batch_size=32)

Epoch 1/100
6/6 [==============================] - 2s 10ms/step - loss: 2.2966 - accuracy: 0.1421
Epoch 2/100
6/6 [==============================] - 0s 11ms/step - loss: 2.2496 - accuracy: 0.2263
Epoch 3/100
6/6 [==============================] - 0s 6ms/step - loss: 2.2120 - accuracy: 0.2737
Epoch 4/100
6/6 [==============================] - 0s 7ms/step - loss: 2.1740 - accuracy: 0.3053
Epoch 5/100
6/6 [==============================] - 0s 14ms/step - loss: 2.1322 - accuracy: 0.3263
Epoch 6/100
6/6 [==============================] - 0s 13ms/step - loss: 2.0880 - accuracy: 0.3474
Epoch 7/100
6/6 [==============================] - 0s 18ms/step - loss: 2.0403 - accuracy: 0.3632
Epoch 8/100
6/6 [==============================] - 0s 35ms/step - loss: 1.9893 - accuracy: 0.4053
Epoch 9/100
6/6 [==============================] - 0s 25ms/step - loss: 1.9343 - accuracy: 0.4211
Epoch 10/100
6/6 [==============================] - 0s 8ms/step - loss: 1.8771 - accuracy: 0.4316
Epoch 11/100
6/6 [====

In [14]:
save_model(hospitalbot, "HospitalBot_v1")

In [15]:
chatbot = load_model("HospitalBot_v1")

In [16]:
tags = [item['tag'] for item in intents['intents']]
tags

['greeting',
 'goodbye',
 'thanks',
 'noanswer',
 'options',
 'adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'search_blood_pressure_by_patient_id',
 'pharmacy_search',
 'search_pharmacy_by_name',
 'hospital_search',
 'search_hospital_by_params',
 'search_hospital_by_type']

In [17]:
resp = [item['responses'] for item in intents['intents']]
resp

[['Hello, thanks for asking',
  'Good to see you again',
  'Hi there, how can I help?'],
 ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
 ['Happy to help!', 'Any time!', 'My pleasure'],
 ["Sorry, can't understand you",
  'Please give me more info',
  'Not sure I understand'],
 ['I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies',
  'Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies'],
 ['Navigating to Adverse drug reaction module'],
 ['Navigating to Blood Pressure module'],
 ['Please provide Patient ID', 'Patient ID?'],
 ['Loading Blood pressure result for Patient'],
 ['Please provide pharmacy name'],
 ['Loading pharmacy details'],
 ['Please provide hospital name or location'],
 ['Please provide hospital type'],
 ['Loading hospital details']]

In [18]:
tag_responses = dict(zip(tags, resp))
tag_responses

{'greeting': ['Hello, thanks for asking',
  'Good to see you again',
  'Hi there, how can I help?'],
 'goodbye': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
 'thanks': ['Happy to help!', 'Any time!', 'My pleasure'],
 'noanswer': ["Sorry, can't understand you",
  'Please give me more info',
  'Not sure I understand'],
 'options': ['I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies',
  'Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies'],
 'adverse_drug': ['Navigating to Adverse drug reaction module'],
 'blood_pressure': ['Navigating to Blood Pressure module'],
 'blood_pressure_search': ['Please provide Patient ID', 'Patient ID?'],
 'search_blood_pressure_by_patient_id': ['Loading Blood pressure result for Patient'],
 'pharmacy_search': ['Please provide pharmacy name'],
 'search_pharmacy_by_name': ['Loading pharmacy details'],
 'hospital_search': ['Please provide hospital name or loc

In [19]:
# json_object = json.dumps(tag_responses, indent=4)

# with open("responses.json", "w") as outfile:
#   outfile.write(json_object)

In [20]:
# Transforming input and predicting intent
def predict_tag(user_input):
  user_input_tfidf = vectorizer.transform([user_input.lower()]).toarray()
  predicted_proba = hospitalbot.predict(user_input_tfidf)
  encoded_label = [np.argmax(predicted_proba)]
  predicted_tag = le.inverse_transform(encoded_label)[0]
  return predicted_tag

In [21]:
# Creating our chat loop
def start_chat():
  print("---------------HospitalBot V1---------------")
  print("Ask any queries!")
  print("Type EXIT to quit\n")
  while True:
    user_input = input("Ask anything: ")
    if user_input == "EXIT":
      time.sleep(1)
      break
    else:
      if user_input:
        tag = predict_tag(user_input)
        response = random.choice(tag_responses[tag])
        time.sleep(1)
        print(response)
      else:
        pass

In [53]:
start_chat(

)

---------------HospitalBot V1---------------
Ask any queries!
Type EXIT to quit

Ask anything: Hello
1/1 [==============================] - 0s 33ms/step
Hello, thanks for asking
Ask anything: sldbnsdlb
1/1 [==============================] - 0s 35ms/step
Good to see you again
Ask anything: pharmacy
1/1 [==============================] - 0s 25ms/step
Please provide pharmacy name
Ask anything: Apoteket
1/1 [==============================] - 0s 22ms/step
Hello, thanks for asking
Ask anything: EXIT


Our model is not the smartest tool in the shed haha. Let's check accuracy!

In [23]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    training_data_tfidf,
    training_data_tags_dummy_encoded,
    test_size=0.2,
    random_state=42
)

# Defining early stopping callback to prevent overfitting by stopping the
# training when the model's performance on a validation set stops improving
early_stopping = EarlyStopping(monitor='val_loss', patience=3,
                               restore_best_weights=True)

# Training the model with early stopping implemented
history = hospitalbot.fit(
    X_train, y_train,
    epochs=50,  # Increase the number of epochs or set it based on your training needs
    batch_size=32,
    verbose=1,
    validation_split=0.1,  # Use a portion of the training data for validation
    callbacks=[early_stopping]
)


Epoch 1/50
5/5 [==============================] - 0s 83ms/step - loss: 0.0756 - accuracy: 0.9559 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 2/50
5/5 [==============================] - 0s 12ms/step - loss: 0.0728 - accuracy: 0.9706 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 3/50
5/5 [==============================] - 0s 16ms/step - loss: 0.0720 - accuracy: 0.9706 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 4/50
5/5 [==============================] - 0s 14ms/step - loss: 0.0718 - accuracy: 0.9706 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 5/50
5/5 [==============================] - 0s 11ms/step - loss: 0.0711 - accuracy: 0.9706 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 6/50
5/5 [==============================] - 0s 16ms/step - loss: 0.0699 - accuracy: 0.9706 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 7/50
5/5 [==============================] - 0s 15ms/step - loss: 0.0721 - accuracy: 0.9706 - val_loss: 0.0020 - val_accuracy: 1.0000


In [24]:
# Evaluating the model on the test set
test_metrics = hospitalbot.evaluate(X_test, y_test, verbose=0)

# Displaying the test accuracy and other metrics
print(f"Test Accuracy: {test_metrics[1]*100:.2f}%")
print(f"Test Loss: {test_metrics[0]}")

# Predicting on the test set
y_pred = hospitalbot.predict(X_test)

# Converting predictions to class labels
y_pred_classes = y_pred.argmax(axis=1)
y_test_classes = y_test.argmax(axis=1)

# Displaying classification report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

Test Accuracy: 89.47%
Test Loss: 0.15313132107257843
2/2 [==============================] - 0s 6ms/step
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         3
           4       0.80      1.00      0.89         4
           5       1.00      1.00      1.00         3
           6       1.00      0.60      0.75        10
           7       1.00      1.00      1.00         3
           9       0.57      1.00      0.73         4

    accuracy                           0.89        38
   macro avg       0.93      0.96      0.93        38
weighted avg       0.93      0.89      0.89        38



Our model is clearly overfitted and has memorized every single response in the JSON file (but somehow still gets it completely wrong sometimes). We want to achieve a balance between training accuracy and generalization

#Let's start on HospitalBot V2!<br>

Apart from improving our model with hyperparameter tuning, we're going to implement the following:
* Context Handling: Utilizing the "context" parameter from the original JSON file to get context-aware responses or prompting for user input
* User Experience and implementing response time based on context
* User Input Preprocessing in the form of:
  * Intent Confidence Threshold: We need to pass a threshold to confidently say “okay, that connects to this tag”. This will almost inevitably be a trade-off between making confident predictions and avoiding incorrect predictions
  * Handling Unknown Inputs and Error Handling: We will introduce default "I don't understand messages" if we don't pass the confidence threshold
  * Fuzzy Matching: The bot will be able to understand slight variations and misspellings of words and phrases


Clarification both for readers and myself on Context Handling: If we write something within the tag of 'blood_pressure_search', the bot will provide a response of either 'Please provide Patient ID?' or simply 'Patient ID?'. But after we have entered that (I'm also now seeing, it shouldn't say "Ask anything" when we enter that as well), it won't go back to its original "Hi there, how can I help?", but rather it will pick the context from the tag "blood_pressure_search", i.e. the current context will be "search_blood_pressure_by_patient_id" and so that will be the new tag and it won't even ask for input but instead give the response in that tag "Loading Blood pressure result for Patient". And from there, it can say "Good luck! I will be here if you need me again" and then shut down or something.<br>
The other key point to this, that just clicked for me, is that we don't predict a tag based on user input if we have a current context!

# Hyperparameter tuning

Uncomment to perform it again. It's commented out to have the entire program fun faster

In [25]:
# # Defining a simple neural network model based on our previous one
# def create_model(optimizer='rmsprop', activation='relu', learning_rate=0.001):
#     model = Sequential()
#     model.add(Dense(10, input_shape=(len(training_data_tfidf[0]),),
#                     activation=activation))
#     model.add(Dense(8, activation=activation))
#     model.add(Dense(8, activation=activation))
#     model.add(Dense(6, activation=activation))
#     model.add(Dense(len(training_data_tags_dummy_encoded[0]),
#                     activation="softmax"))

#     # Use the specified optimizer and learning rate
#     if optimizer == 'adam':
#         opt = Adam(learning_rate=learning_rate)
#     elif optimizer == 'sgd':
#         opt = SGD(learning_rate=learning_rate)
#     else:
#         opt = optimizer  # Use the default optimizer if specified

#     model.compile(optimizer=opt, loss="categorical_crossentropy",
#                   metrics=["accuracy"])
#     return model

# # Wrap the Keras model as an estimator; KerasClassifier is a convenient
# wrapper allowing us to use a Keras neural network model as if it was a
# classifier in the Scikit-Learn framework. The RandomizedSearchCV kinda
# needs a familiar estimator to use its functions and so we use the
# KerasClassifier to kinda mimic that behavior; to use our Keras model within
# this Scikit-Learn context
# estimator = KerasClassifier(build_fn=create_model, activation='relu',
#                             epochs=10, batch_size=32, learning_rate=0.001,
#                             verbose=0)


# # Define hyperparameters and their distributions for RandomizedSearchCV
# param_dist = {
#     'optimizer': [Adam(), SGD()],
#     'activation': ['relu', 'sigmoid'],
#     'batch_size': [16, 32, 64],
#     'epochs': [5, 10, 15],
#     'learning_rate': uniform(loc=0.0001, scale=0.1),
# }


# # Perform random search
# random_search = RandomizedSearchCV(estimator=estimator,
#                                    param_distributions=param_dist,
#                                    scoring='accuracy', cv=3, n_iter=10)
# random_search_result = random_search.fit(X_train, y_train)

# # Print the best hyperparameters
# print("Best: %f using %s" % (random_search_result.best_score_,
#                              random_search_result.best_params_))

In [26]:
# Let's create a DNN with our best parameters!
best_params = {'activation': 'relu',
               'batch_size': 16,
               'epochs': 15,
               'learning_rate': 0.07242196012176288,
               'optimizer': 'SGD'}

hospitalbot_v2 = Sequential()
hospitalbot_v2.add(Dense(10, input_shape=(len(training_data_tfidf[0]),)))
hospitalbot_v2.add(Dense(8))
hospitalbot_v2.add(Dense(8))
hospitalbot_v2.add(Dense(6))
hospitalbot_v2.add(Dense(len(training_data_tags_dummy_encoded[0]),
                      activation="softmax"))
hospitalbot_v2.compile(optimizer="SGD", loss="categorical_crossentropy",
                    metrics="accuracy")

hospitalbot_v2.fit(training_data_tfidf, training_data_tags_dummy_encoded,
                epochs=15, batch_size=16)

Epoch 1/15
12/12 [==============================] - 1s 2ms/step - loss: 2.3388 - accuracy: 0.1158
Epoch 2/15
12/12 [==============================] - 0s 2ms/step - loss: 2.3307 - accuracy: 0.1053
Epoch 3/15
12/12 [==============================] - 0s 2ms/step - loss: 2.3231 - accuracy: 0.0947
Epoch 4/15
12/12 [==============================] - 0s 3ms/step - loss: 2.3157 - accuracy: 0.1158
Epoch 5/15
12/12 [==============================] - 0s 2ms/step - loss: 2.3089 - accuracy: 0.1158
Epoch 6/15
12/12 [==============================] - 0s 3ms/step - loss: 2.3026 - accuracy: 0.1105
Epoch 7/15
12/12 [==============================] - 0s 2ms/step - loss: 2.2960 - accuracy: 0.1263
Epoch 8/15
12/12 [==============================] - 0s 2ms/step - loss: 2.2899 - accuracy: 0.1263
Epoch 9/15
12/12 [==============================] - 0s 2ms/step - loss: 2.2847 - accuracy: 0.1421
Epoch 10/15
12/12 [==============================] - 0s 4ms/step - loss: 2.2792 - accuracy: 0.1316
Epoch 11/15
12/12 [

In [27]:
save_model(hospitalbot_v2, "HospitalBot_v2")

In [28]:
chatbot = load_model("HospitalBot_v2")

With our new chatbot, below are all of the new chat functionality. I did these before doing the Hyperparameter tuning haha but I moved it around now in the notebook to represent the optimal workflow that I should have implemented

In [29]:
tags = [item['tag'] for item in intents['intents']]
contexts = [item['context'] for item in intents['intents']]
tag_contexts = dict(zip(tags, contexts))
tag_contexts

{'greeting': [''],
 'goodbye': [''],
 'thanks': [''],
 'noanswer': [''],
 'options': [''],
 'adverse_drug': [''],
 'blood_pressure': [''],
 'blood_pressure_search': ['search_blood_pressure_by_patient_id'],
 'search_blood_pressure_by_patient_id': [''],
 'pharmacy_search': ['search_pharmacy_by_name'],
 'search_pharmacy_by_name': [''],
 'hospital_search': ['search_hospital_by_params'],
 'search_hospital_by_params': ['search_hospital_by_type'],
 'search_hospital_by_type': ['']}

In [30]:
# We will use an updated predict_tag function that implements Intent
# Confidence Threshold
def predict_tag_v2(user_input, confidence_threshold=0.7):
    user_input_tfidf = vectorizer.transform([user_input.lower()]).toarray()
    #print(f"user_input_tfidf: {user_input_tfidf}") For debugging purposes

    predicted_proba = hospitalbot.predict(user_input_tfidf)
    #print(f"predicted_proba: {predicted_proba}") For debugging purposes

    max_confidence = np.max(predicted_proba)

    if max_confidence >= confidence_threshold:
        encoded_label = [np.argmax(predicted_proba)]
        predicted_tag = le.inverse_transform(encoded_label)[0]
        #print(f"predicted_tag: {predicted_tag}") For debugging purposes
        return predicted_tag
    else:
        # If confidence is below the threshold, return a special tag for
        # unknown input
        return "unknown_input"


# We will also implement fuzzy matching so that it not only can pick up on
# "hospital" and associate it with the "hospital_search" tag, but also any
# slight variation or misspelling of it
# def fuzzy_match(user_input, confidence_threshold=0.8):
#     valid_matches = ["blood", "pharmacy", "hospital"]
#     fuzzy_match = [valid_match for valid_match in valid_matches if fuzz.partial_ratio(user_input, valid_match) >= confidence_threshold]

#     if fuzzy_match:
#       return fuzzy_match[0]
#     else:
#       return "unknown_input"

# Updated version where we define the order of valid matches and select
# the one with the highest fuzzy match ratio. We also have an unknown
# threshold so that the bot will still respond with a version of
# "I didn't understand that" if we are typing in gibberish
def fuzzy_match(user_input, confidence_threshold=0.8, unknown_threshold=70):
    valid_matches = ["hospital", "pharmacy", "blood", "blood pressure",
                     "adverse drug", "hi", "hello", "help"]

    # Initialize variables to store the best match and its ratio
    best_match = None
    best_ratio = 0

    for valid_match in valid_matches:
        if valid_match.lower() in user_input.lower():  # Exact match for
        #certain keywords
            ratio = 100
        else:
            ratio = fuzz.partial_ratio(user_input, valid_match)

        # Check if the current match has a higher ratio than the best match
        if ratio >= confidence_threshold and ratio > best_ratio:
            best_match = valid_match
            best_ratio = ratio

    if best_match and best_ratio >= unknown_threshold:
        return best_match
    else:
        return "unknown_input"

In [31]:
# Apart from just the tag, we will also consider the current context for
# more context-aware responses and staying on topic
default_messages = ["I didn't catch that. Can you please try again?",
                    "I'm sorry, I didn't understand that. Can you please provide more information?",
                    "I'm not sure I understand. Can you help me understand you better?"]

def generate_response(input_tag, input_context):
    if input_tag == "unknown_input":
      # If the chatbot doesn't understand, it will display one of our
      # variations of "I don't understand"
        return random.choice(default_messages)
    elif input_context is not None and len(input_context) > 2:
        # If we have a context, use it to fetch responses
        return random.choice(tag_responses.get(input_context, [""]))
    #elif len(input_tag) > 2:
    else:
        # If no context, use the tag to fetch responses
        return random.choice(tag_responses.get(input_tag, [""]))

# Logic to update the current context based on the current tag
# We return an empty string as a default value is the specified key
# is not found
def update_context(input_tag):
    global current_context
    new_context = tag_contexts.get(input_tag, [""])[0]
    current_context = new_context if len(new_context) > 2 else ""

In [32]:
def one_letter_at_a_time(output_string:str):
    for char in output_string:
      print(char, end='')
      sys.stdout.flush()
      time.sleep(0.03)

ending_message = "Good luck! I will be here if you need me again!"
def end_chatbot():
    time.sleep(2)
    one_letter_at_a_time("•\n•\n•\n•\n")
    time.sleep(2)
    one_letter_at_a_time(ending_message)
    time.sleep(1)

In [54]:
current_context = ""  # Initializing current context

def start_chat_v2():
    global current_context
    print("---------------HospitalBot V2---------------")
    print("Ask any queries!")
    print("Type EXIT to quit\n")
    while True:
        user_input = input("Type here: ")
        if user_input == "EXIT":
            time.sleep(1)
            break
        else:
            if user_input:
                # print(f"Current context: {current_context}")
                # For debugging purposes

                # If we have a current context, the tag is grabbed based on
                # the context in the generate_response function
                if current_context is not None and len(current_context) > 2:
                    tag = current_context
                else:
                    # If it's the first time in the chat loop or if we do not have
                    # a current context, we predict the tag based on user input,
                    # after a fuzzy match check
                    fuzzy_check = fuzzy_match(user_input.lower())
                    tag = predict_tag_v2(fuzzy_check)

                # We generate a response with both the current tag and
                # context in mind (if we have one)
                response = generate_response(tag, current_context)

                # Update context based on the current tag
                update_context(tag) # I had this as an assignment which
                # returned None and it was driving me crazy for an hour
                current_context = current_context
                time.sleep(1)

                # Below are the tags that end the conversation
                # I tried a lot of solutions but I opted for this rather
                # naïve looking one where we have branches of conversation
                # based on the tag. This works because these context-specific
                # tags have no patterns leading to them!
                if tag == 'search_blood_pressure_by_patient_id':
                  one_letter_at_a_time(f"{response} #{user_input}\n")
                  end_chatbot()
                  break
                elif tag == 'search_pharmacy_by_name':
                  one_letter_at_a_time(f"{response} for {user_input}\n")
                  end_chatbot()
                  break

                # This one is a bit special in that it doesn't end the chatbot
                # Instead it saves the Hospital name/location in order to
                # display it in the next step of the context interaction
                elif tag == 'search_hospital_by_params':
                  hospital_param = user_input
                  one_letter_at_a_time(response + "\n")
                  time.sleep(1)
                elif tag == 'search_hospital_by_type':
                  if 'hospital' in hospital_param.lower():
                    one_letter_at_a_time(f"{response} for {hospital_param}\n")
                  else:
                    one_letter_at_a_time(f"{response} for {hospital_param} {user_input} Hospital\n")
                  end_chatbot()
                  break

                else:
                  one_letter_at_a_time(response + "\n")
                  time.sleep(1)
            else:
                pass

start_chat_v2()

---------------HospitalBot V2---------------
Ask any queries!
Type EXIT to quit

Type here: oawuevsjdn
1/1 [==============================] - 0s 25ms/step
I'm not sure I understand. Can you help me understand you better?
Type here: Duck
1/1 [==============================] - 0s 24ms/step
I'm not sure I understand. Can you help me understand you better?
Type here: Hello
1/1 [==============================] - 0s 27ms/step
Hi there, how can I help?
Type here: hsptak
1/1 [==============================] - 0s 26ms/step
I'm sorry, I didn't understand that. Can you please provide more information?
Type here: hosptak
1/1 [==============================] - 0s 24ms/step
Please provide hospital name or location
Type here: Stockholm City
Please provide hospital type
Type here: Regional
Loading hospital details for Stockholm City Regional Hospital
•
•
•
•
Good luck! I will be here if you need me again!

Let's check accuracy!

In [34]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    training_data_tfidf,
    training_data_tags_dummy_encoded,
    test_size=0.2,
    random_state=42
)

# Defining early stopping callback to prevent overfitting by stopping the
# training when the model's performance on a validation set stops improving
early_stopping = EarlyStopping(monitor='val_loss', patience=3,
                               restore_best_weights=True)

# Training the model with early stopping implemented
history = hospitalbot_v2.fit(
    X_train, y_train,
    epochs=50,  # Increase the number of epochs or set it based on your training needs
    batch_size=32,
    verbose=1,
    validation_split=0.1,  # Use a portion of the training data for validation
    callbacks=[early_stopping]
)


Epoch 1/50
5/5 [==============================] - 0s 64ms/step - loss: 2.2524 - accuracy: 0.1324 - val_loss: 2.2321 - val_accuracy: 0.1250
Epoch 2/50
5/5 [==============================] - 0s 11ms/step - loss: 2.2494 - accuracy: 0.1324 - val_loss: 2.2302 - val_accuracy: 0.1250
Epoch 3/50
5/5 [==============================] - 0s 13ms/step - loss: 2.2472 - accuracy: 0.1324 - val_loss: 2.2264 - val_accuracy: 0.1875
Epoch 4/50
5/5 [==============================] - 0s 15ms/step - loss: 2.2438 - accuracy: 0.1618 - val_loss: 2.2238 - val_accuracy: 0.1875
Epoch 5/50
5/5 [==============================] - 0s 15ms/step - loss: 2.2414 - accuracy: 0.1838 - val_loss: 2.2223 - val_accuracy: 0.1875
Epoch 6/50
5/5 [==============================] - 0s 15ms/step - loss: 2.2384 - accuracy: 0.1838 - val_loss: 2.2210 - val_accuracy: 0.1875
Epoch 7/50
5/5 [==============================] - 0s 11ms/step - loss: 2.2359 - accuracy: 0.1838 - val_loss: 2.2186 - val_accuracy: 0.1875
Epoch 8/50
5/5 [===========

In [35]:
# Evaluating the model on the test set
test_metrics = hospitalbot_v2.evaluate(X_test, y_test, verbose=0)

# Displaying the test accuracy and other metrics
print(f"Test Accuracy: {test_metrics[1]*100:.2f}%")
print(f"Test Loss: {test_metrics[0]}")

# Predicting on the test set
y_pred = hospitalbot_v2.predict(X_test)

# Converting predictions to class labels
y_pred_classes = y_pred.argmax(axis=1)
y_test_classes = y_test.argmax(axis=1)

# Displaying classification report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

Test Accuracy: 18.42%
Test Loss: 2.1479597091674805
2/2 [==============================] - 0s 5ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         3
           4       0.14      1.00      0.25         4
           5       0.25      0.33      0.29         3
           6       1.00      0.20      0.33        10
           7       0.00      0.00      0.00         3
           9       0.00      0.00      0.00         4

    accuracy                           0.18        38
   macro avg       0.15      0.17      0.10        38
weighted avg       0.30      0.18      0.14        38



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


So the model actually performs significantly worse after the Hyperparameter tuning, hence why I now retroactively have chosen to run with the first model for the chatbot. I tried running the chat loop with the V2 DNN and it couldn't understand anything, it couldn't pick up a single tag.<br>
Why does it perform worse after the Hyperparameter tuning? Let's explore a few reasons why this might be the case:
*   The hyperparameter search space might have been too broad, and random search might have explored combinations that do not work well together.
*   Since the dataset is rather small (and I have chosen to not add anything to the dataset that we are given), hyperparameter tuning might lead to models that are too complex and perform poorly on new data.
*   The metric used for hyperparameter tuning might not have aligned with the actual goal of the application.

Still, I made a rather functioning chatbot with the first DNN in combination with functionality for the chat loop such as context handling and fuzzy matching, and I am satisfied with the results! It doesn't understand "hsptl" and it doesn't understand "hostpl", but it does understand "hosptl". That is a fine line of understanding that I am okay with<br><br>
Further improvements that I stumbled upon but have chosen not to implement include:
*   Natural Language Understanding (NLU): We could incorporate pre-trained language models like BERT.
*   Logging and Analytics: We could implement logging to capture all user queries entered, predicted intents, and bot responses. This can provide valuable insights into user interactions and areas for improvement.
*   If wanted to actually deploy the chatbot, we would have to ensure that it can handle concurrent users and is scalable. We would consider deploying it on a platform that supports the desired level of usage.





Below is a dump and a code graveyard of all old and brancing code snippets from endeavors of trying to make things like the context handling work. A lot of it is written by ChatGPT by having a collaborative conversation with it sharing my vision of my output. At one point it suggested a Dictionary to track context-specific flags and at that point I tapped out and went to bed. I woke up the next morning and fixed it on my own within the first 20 minutes of coding! (I cannot I prove it but I swear I am not lying when I say that haha)

In [36]:
# # def generate_response(input_tag, input_context):
# #   global current_context
# #   # If there is no current context, we simply grab a response from the
# #   # tag dictionary and return it. Similar to the first version of the
# #   # chatbot; we can even copy the cope from it
# #   if len(input_context) < 2:
# #     return random.choice(tag_responses[input_tag])

# #   # If there is current context, the chatbot should behave differently and
# #   # more aware.
# #   else:
# #     new_tag = tag_contexts[input_tag]
# #     return random.choice(tag_responses[new_tag])

# def update_context(input_tag):
#     global current_context
#     new_context = tag_contexts.get(input_tag, [""])[0]
#     current_context = new_context if new_context is not None and len(new_context) > 2 else ""
#     print(f"Updated context to: {current_context}")

# def update_context(input_tag):
#     global current_context
#     global end_of_interaction

#     new_context = tag_contexts.get(input_tag, [""])[0]
#     current_context = new_context if len(new_context) > 2 else ""

#     print(f"New context before update: {new_context}")  # Add this line for debugging
#     # Set end_of_interaction to True if the new_context is not empty
#     end_of_interaction = bool(new_context)
#     print(f"Updated context to: {current_context}")  # Add this line for debugging
#     print(f"End of interaction: {end_of_interaction}")  # Add this line for debugging

# def generate_response(input_tag, input_context):
#     global end_of_interaction

#     # Logic to generate responses based on the tag and current context
#     # Use the intents JSON to determine appropriate responses
#     if input_context is not None and len(input_context) > 2:
#         # If we have a context, use it to fetch responses.
#         response = random.choice(tag_responses.get(input_context, [""]))
#         # Switch the global end_of_interaction boolean to True
#         end_of_interaction = True
#         return response
#     elif len(input_tag) > 2:
#         # If no context, use the tag to fetch responses
#         return random.choice(tag_responses.get(input_tag, [""]))
#     else:
#         # If nothing works, we chatbot will simply say it doesn't
#         # understand
#         return "Sorry, I couldn't understand that."

# def generate_response(input_tag, input_context):
#     # Define context-specific actions
#     context_actions = {
#         "search_blood_pressure_by_patient_id": handle_blood_pressure_by_patient_id,
#         # Add other context-action mappings as needed
#     }

#     # Check if the input_context has a corresponding action
#     if input_context in context_actions:
#         # Execute the context-specific action and get the response
#         response = context_actions[input_context](input_tag)
#         return response
#     elif len(input_tag) > 2:
#         # If no context, use the tag to fetch responses
#         return random.choice(tag_responses.get(input_tag, [""]))
#     else:
#         # If nothing works, the chatbot will simply say it doesn't understand
#         return "Sorry, I couldn't understand that."

# # Dictionary to track context-specific flags
# context_flags = {
#     "search_blood_pressure_by_patient_id": False,
#     # Add other contexts as needed
# }

# # Example context-specific action for "search_blood_pressure_by_patient_id"
# def handle_blood_pressure_by_patient_id(input_tag):
#     global context_flags

#     # Implement the specific logic for this context
#     response = random.choice(tag_responses.get(input_context, [""]))

#     # Check if the response contains "Please provide Patient ID" and set the flag accordingly
#     context_flags[input_context] = "Please provide Patient ID" in response

#     return response



# def update_context(input_tag):
#     global current_context
#     global end_of_interaction

#     new_context = tag_contexts.get(input_tag, [""])[0]
#     current_context = new_context if len(new_context) > 2 else ""

#     # Set end_of_interaction to True if the current_context indicates the end
#     end_of_interaction = "Good luck!" in current_context


# def start_chat_v2():
#     global current_context
#     global end_of_interaction

#     print("---------------HospitalBot V2---------------")
#     print("Ask any queries!")
#     print("Type EXIT to quit\n")

#     while True:
#       user_input = input("Enter here: ")

#       if user_input == "EXIT":
#           time.sleep(1)
#           break

#       if user_input:
#           print(f"Current context: {current_context}")
#           print(f"End of interaction: {end_of_interaction}")

#           if len(current_context) < 2:
#               tag = predict_tag(user_input)
#               response = generate_response(tag, current_context)
#               print(response)
#               current_context = update_context(tag)

#               # Check if we are waiting for specific information based on the context
#               if current_context is not None and not context_flags.get(current_context, False):
#                   continue  # Continue the loop to prompt for information

#               # Check if the current context is empty and if end_of_interaction is True
#               if not current_context and end_of_interaction:
#                   print("Good luck! I will be here if you need me again")
#                   time.sleep(1)
#                   print("Breaking out of the loop")
#                   break
#           else:
#               pass